In [0]:
# Este código se ejecuta en una celda de un Notebook de Databricks
# Su responsabilidad es crear la tabla de la Capa de Oro (Gold),
# detectando automáticamente la tabla de plata más reciente para procesar.

# 1. --- Detección Automática de la Tabla de Plata ---
# En lugar de definir un nombre de tabla fijo, consultamos el information_schema
# para encontrar la tabla más reciente en nuestro schema de datos procesados
# que siga el patrón de nomenclatura '_cleaned'.

silver_schema = "workspace.tecnomundo_data_processed"
dimension_table = "workspace.tecnomundo_data_dimensions.category"

print("Iniciando la detección automática de la tabla de plata más reciente...")

try:
    # Esta consulta busca en los metadatos de Unity Catalog
    latest_silver_table_df = spark.sql(f"""
        SELECT CONCAT(table_catalog, '.', table_schema, '.', table_name) as full_table_name
        FROM system.information_schema.tables
        WHERE table_schema = '{silver_schema.split('.')[-1]}'
          AND table_catalog = '{silver_schema.split('.')[0]}'
          AND table_name LIKE '%_cleaned'
        ORDER BY last_altered DESC
        LIMIT 1
    """)

    if latest_silver_table_df.count() == 0:
        raise ValueError(f"No se encontró ninguna tabla con el sufijo '_cleaned' en el schema '{silver_schema}'.")

    # Extraemos el nombre completo de la tabla del resultado de la consulta
    silver_table = latest_silver_table_df.first().full_table_name
    print(f"Tabla de plata detectada para procesar: {silver_table}")

except Exception as e:
    print(f"Error durante la detección de la tabla: {e}")
    # Detenemos la ejecución si no se puede encontrar la tabla de origen
    dbutils.notebook.exit("No se pudo detectar la tabla de origen de la capa de plata.")


# --- Generación Dinámica del Nombre de la Tabla de Oro ---
silver_table_base_name = silver_table.split('.')[-1]
output_gold_schema = ".".join(silver_table.split('.')[:-1])

if silver_table_base_name.endswith('_cleaned'):
    clean_base_name = silver_table_base_name[:-8]
else:
    clean_base_name = silver_table_base_name

gold_table = f"{output_gold_schema}.{clean_base_name}_categorized"


print(f"\nTabla de Origen (Plata): {silver_table}")
print(f"Tabla de Dimensiones: {dimension_table}")
print(f"Tabla de Destino (Oro): {gold_table}")


# 2. --- Lectura de las Tablas de Origen ---
print(f"\nLeyendo datos desde la Capa de Plata: '{silver_table}'...")
df_silver = spark.table(silver_table)

print(f"Leyendo datos desde la tabla de Dimensiones: '{dimension_table}'...")
df_dimensions = spark.table(dimension_table)

# --- PASO DE DEPURACIÓN: Imprimir el esquema de la tabla de dimensiones ---
# Este comando nos mostrará los nombres exactos de las columnas en la tabla 'category'.
print("\nEsquema de la tabla de Dimensiones ('category'):")
df_dimensions.printSchema()
# --------------------------------------------------------------------


# 3. --- Enriquecimiento de Datos (Categorización) ---
print("\nEnriqueciendo datos con la tabla de categorías...")

# Se usa el nombre de columna saneado correcto 'comprobante_n'
df_gold = df_silver.join(
    df_dimensions,
    df_silver.cdigo == df_dimensions.cdigo_interno,
    "left"
).select(
    df_silver.fecha,
    df_silver.comprobante_n,
    df_silver.cdigo.alias("codigo_producto"),
    df_dimensions.nombre_del_producto, # <-- Este nombre podría ser incorrecto
    df_dimensions.categoria,           # <-- Este nombre podría ser incorrecto
    df_silver.cantidad,
    df_silver.precio_un_,
    df_silver.ganancia,
    df_silver.subtotal
)

# Se usa el nombre de columna saneado correcto 'comprobante_n' para renombrar
df_gold = (df_gold
           .withColumnRenamed("comprobante_n", "numero_comprobante")
           .withColumnRenamed("precio_un_", "precio_unitario"))

print("Enriquecimiento completado.")


# 4. --- Guardado de la Tabla Final (Capa de Oro) ---
print(f"\nGuardando la tabla final enriquecida en la Capa de Oro: '{gold_table}'...")
(df_gold.write
 .mode("overwrite")
 .saveAsTable(gold_table))
print("¡Tabla de la Capa de Oro guardada exitosamente!")


# 5. --- Verificación Final ---
print("\nMostrando una muestra de la tabla final (Capa de Oro):")
display(spark.table(gold_table))

